In [ ]:
import pymysql

ENDPOINT = '10.10.211.59'
PORT = 3306
USERNAME = 'read_only'
DBNAME = 'CAPC_ACCURACY_REPORT'
PASSWORD = 'P@ssw0rd@123' # Change this based on your own password
CURSORCLASS = pymysql.cursors.DictCursor

In [ ]:
def initiate_local_connection():
    try:
        connection = pymysql.connect(host=ENDPOINT,
                                     port=PORT,
                                     user=USERNAME,
                                     passwd=PASSWORD,
                                     db=DBNAME,
                                     cursorclass=CURSORCLASS)
        print('[+] Local Connection Successful')
    except Exception as e:
        print(f'[+] Local Connection Failed: {e}')
        connection = None

    return connection

In [ ]:
#  Call function and return the connection object
connection = initiate_local_connection()

In [ ]:
query = '''select * from overall_data where month in ('2023-07') and client_code='CHS';'''

In [ ]:
df_query = pd.read_sql(query, con=connection)

In [ ]:
df_query

In [ ]:
df_query.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_Overall_sheet_till_11th.xlsx')

In [ ]:
df = df_query.copy()

In [ ]:
from collections import OrderedDict
import pandas as pd
def get_accuracy(data_df, codetype):
    
    # display(data_df)
    # print(data_df.columns)
    
    accept = data_df[codetype+'_accepted_count'].sum()
    reject = data_df[codetype+'_rejected_count'].sum()
    add = data_df[codetype+'_added_count'].sum()
    
    d = OrderedDict()
    # d['facility_name'] = data_df['facility_name'].iloc[0]
    d['encounters'] = data_df.shape[0]
    
    d[codetype+'_accepted'] = accept
    d[codetype+'_rejected'] = reject
    d[codetype+'_added'] = add
    
    d[codetype+'_billed'] = accept+add
    
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [ ]:
df.groupby(['facility_code']).apply(get_accuracy, codetype='eandm').sort_values(['facility_code'],ascending=[True])

In [ ]:
df.enc_doc_id = df.enc_doc_id.astype('str')

In [ ]:
df.shape

In [ ]:
df_single_docs = df[~df.enc_doc_id.str.contains(',')]

In [ ]:
df[df.enc_visit_type.str.contains('RECHECK')][cols_to_use]

In [ ]:
df_single_docs.shape

In [ ]:
df_9954 = df[df['facility_code'] == '9954']

In [ ]:
df_9954.shape

In [ ]:
df_9954.groupby(['enc_visit_type']).apply(get_accuracy, codetype='eandm').sort_values(['eandm_rejected'],ascending=[False])

In [ ]:
df_9954.enc_visit_type = df_9954.enc_visit_type.astype('str')

In [ ]:
df.columns

In [ ]:
cols_to_use = ['encounter_id', 'service_date', 'enc_visit_type', 'enc_place_of_service', 'facility_code', 'account_number', 'eandm_suggested', 'eandm_final', 'cpt_suggested', 'cpt_final']

In [ ]:
df_9954[df_9954.enc_visit_type.str.contains('N/A')][cols_to_use]

In [1]:
import pandas as pd

In [24]:
df = pd.read_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_Overall_sheet_till_11th.xlsx', index_col=False)

In [25]:
df.facility_code = df.facility_code.astype('str')

In [26]:
df_1575 = df[df['facility_code'] == '1575']

In [27]:
df_1575[cols_to_use]

,encounter_id,service_date,enc_visit_type,enc_place_of_service,facility_code,account_number,cpt_final,cpt_suggested,eandm_final,eandm_suggested,local_suggested_ruleEngine,eandm_accepted_count,eandm_rejected_count,eandm_added_count,local_suggested_14_july,new,17_july_head,New_4
440,8262270,2023-07-07 09:15:00,*Follow-up 15,OV,1575,5054338,"51700, 1036F","{51700=[], 1036F=[]}",99214 = [25-Sig/Sep ID E&M Same Pro/Day],{99214=[25]},99214,1,0,0,99214,99214,99214,99214
441,8269436,2023-07-06 13:15:00,*Follow-up 15,OV,1575,4986778,"3008F, 1170F, 1160F, 11719, 1126F, 1159F, 1036F","{3008F=[], 1170F=[], 1160F=[], 1126F=[], 1159F...",99213 = [25-Sig/Sep ID E&M Same Pro/Day],{99212=[]},99213,0,1,1,99213,99213,99213,99213
442,8163961,2023-07-03 09:30:00,*New Patient 15,OV,1575,5237054,"20610 = [LT-Left Side], 3008F, 1036F, 73030 = ...","{20610=[LT], 3008F=[], 1036F=[], 73030=[LT]}",99203 = [25-Sig/Sep ID E&M Same Pro/Day],{99203=[25]},99203,1,0,0,99203,99203,99203,99203
443,8243322,2023-07-06 15:00:00,*New Patient 30,OV,1575,5236500,"51700, 1035F, 1036F, 1034F","{85520=[], 51700=[], 1035F=[], 1036F=[], 1034F...",99204 = [25-Sig/Sep ID E&M Same Pro/Day],{99203=[25]},99203,0,1,1,99203,99203,99203,99204
444,8163963,2023-07-03 10:30:00,*Follow-up 15,OV,1575,5171496,"1160F, 1125F, 73562 = [LT-Left Side], 1159F, 1...","{1160F=[], 1125F=[], 73562=[LT], 1159F=[], 103...",99213,{99213=[]},99213,1,0,0,99213,99213,99213,99213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,8091525,2023-06-29 09:45:00,*Follow-up 15,OV,1575,4982980,"1160F, 1125F, 1159F, 1035F, 1034F","{20610=[50], 1160F=[], 1125F=[], 1159F=[], 103...",99214,{99214=[25]},99214,1,0,0,99214,99214,99214,99214
752,8123781,2023-06-26 15:30:00,*New Patient 30,OV,1575,4986085,"3008F, 73630 = [RT-Right Side], 1036F, 28470 =...","{3008F=[], 73630=[RT], 28470=[54], 1036F=[]}",99204 = [25-Sig/Sep ID E&M Same Pro/Day],{99203=[]},99203,0,1,1,99203,99203,99203,99203
753,8250242,2023-07-05 14:00:00,*New Patient 30,OV,1575,4993540,NaN,{},99204,{99203=[25]},99203,0,1,1,99203,99203,99203,99203
754,8237442,2023-07-06 11:30:00,*New Patient 30,OV,1575,5156198,"1160F, 1159F, 3077F, 1036F, 3078F, 93000","{1160F=[], 1159F=[], 3077F=[], 1036F=[], 93000...",99205 = [25-Sig/Sep ID E&M Same Pro/Day],"{99204=[25, 95]}",99204,0,1,1,99204,99204,99204,99204


In [30]:
cs_1575 = pd.crosstab(df_1575['clean_final'], df_1575['clean_suggested'], margins=True)

In [29]:
import re
for i,r in df_1575.iterrows():
    df_1575.loc[i,'clean_suggested'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_suggested']))])
    df_1575.loc[i,'clean_final'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_final']))])

/tmp/ipykernel_96944/3804156787.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1575.loc[i,'clean_suggested'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_suggested']))])
/tmp/ipykernel_96944/3804156787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1575.loc[i,'clean_final'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_final']))])


In [ ]:
cs_1575.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_ALL_TARS/1902.xlsx')

In [ ]:
# df_1570[(df_1570.clean_final == '99213') & (df_1570.clean_suggested == '99212')][cols_to_use].to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_ALL_TARS/docs_1570_JULY/Output/TEST.xlsx')

In [ ]:
df_1575[(df_1575.clean_final == '99203') & (df_1575.clean_suggested == '99202')][cols_to_use]

In [33]:
df_1575.cpt_final = df_1575.cpt_final.astype('str')

/tmp/ipykernel_96944/80691998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1575.cpt_final = df_1575.cpt_final.astype('str')


In [35]:
df_1575[df_1575.cpt_final.str.contains('73630')][cols_to_use].eandm_final.value_counts()

99204                                       4
99204 = [25-Sig/Sep ID E&M Same Pro/Day]    3
99203                                       1
99214                                       1
99203 = [25-Sig/Sep ID E&M Same Pro/Day]    1
99213 = [25-Sig/Sep ID E&M Same Pro/Day]    1
99213                                       1
Name: eandm_final, dtype: int64

In [ ]:
df_1575[~df_1575.clean_suggested.str.contains('9') & ((df_1575.clean_final.str.contains('9')))][cols_to_use]

In [ ]:
# .to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_ALL_TARS/docs_1570_JULY/Output/TEST.xlsx')

In [3]:
cols_to_use = ['encounter_id', 'service_date', 'enc_visit_type', 'enc_place_of_service', 'facility_code', 'account_number', 'cpt_final', 'cpt_suggested','eandm_final', 'eandm_suggested','local_suggested_ruleEngine','eandm_accepted_count',
       'eandm_rejected_count', 'eandm_added_count', 'local_suggested_14_july', 'new', '17_july_head', 'New_4']

In [4]:
df_copy = df.copy()

In [21]:
df_copy = df_copy[cols_to_use]

In [6]:
df_copy.local_suggested_ruleEngine = df_copy.local_suggested_ruleEngine.astype('str')

In [7]:
df_copy = df_copy[~df_copy.local_suggested_ruleEngine.str.contains('vlookup_null')]

In [8]:
# df_copy.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/local_JULY.xlsx')

In [9]:
import re
for i,r in df_copy.iterrows():
    df_copy.loc[i,'clean_suggested'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_suggested']))])
    df_copy.loc[i,'clean_final'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_final']))])

In [10]:
df_copy

,encounter_id,service_date,enc_visit_type,enc_place_of_service,facility_code,account_number,cpt_final,cpt_suggested,eandm_final,eandm_suggested,local_suggested_ruleEngine,eandm_accepted_count,eandm_rejected_count,eandm_added_count,local_suggested_14_july,new,17_july_head,New_4,clean_suggested,clean_final
0,8235901,2023-07-06 11:00:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095114,85610,"{85610=[QW], 1036F=[]}",NaN,{},99212,0,0,0,99212,99212,99212,99212,,
1,8256124,2023-07-06 13:30:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095124,85610,"{1170F=[], 1160F=[], 85610=[QW], 1159F=[], 103...",NaN,{},99213,0,0,0,99213,99213,99213,99213,,
2,8256117,2023-07-06 13:15:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8803379,85610,"{85610=[QW], 1035F=[], 1034F=[]}",NaN,{},99212,0,0,0,99212,99212,99212,99212,,
3,8229493,2023-07-05 14:00:00,*Hosp Follow-up 30,Office or Other Outpatient Visit,3038,8057412,"3008F, 93040, 3077F, 1036F, 3078F","{3008F=[], 93040=[], 3077F=[], 1036F=[], 3078F...",99214,{99214=[25]},99214,1,0,0,99214,99214,99214,99214,99214,99214
4,8234612,2023-07-06 09:30:00,*Follow-up 15,Office or Other Outpatient Visit,3038,8057430,"1160F, 93040, 1159F, 1126F, 1036F","{1160F=[], 93040=[], 1159F=[], 1126F=[], 1036F...",99214,{99214=[25]},99214,1,0,0,99214,99214,99214,99214,99214,99214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8414,8228747,2023-07-05 14:00:00,*Follow-up 15,OV,1523,4060723,NaN,{81001=[]},99213,{99213=[]},99213,1,0,0,99213,99213,99213,99213,99213,99213
8415,8082306,2023-06-20 13:00:00,uroflow/PVR,OV,1523,4329261,"1160F, 1159F, 1036F","{1160F=[], 1159F=[], 1036F=[]}",99214,{99214=[25]},99214,1,0,0,99214,99214,99214,99214,99214,99214
8416,8213382,2023-07-05 13:30:00,ER FOLLOW-UP,OV,1523,4351334,"1160F, 1159F, 1036F","{3008F=[], 1160F=[], 1159F=[], 87210=[QW], 103...",99213,{99213=[]},99213,1,0,0,99213,99213,99213,99213,99213,99213
8417,8113030,2023-06-30 11:30:00,ESTABLISHED PATIENT,OV,1523,4349846,81025,{81025=[]},99213,{99213=[]},99213,1,0,0,99213,99213,99213,99213,99213,99213


In [11]:
df_copy.clean_final.value_counts()

99213           2313
                2101
99214           1471
99203            842
99204            556
99212            533
99396            110
99395            103
99202             95
99385             52
99386             52
99211             37
99442             33
99215             24
99205             12
99213, 99396       9
99406, 99213       9
99441              8
99397              8
99244              7
99406, 99214       6
99243              5
99406, 99204       5
99203, 99386       4
99395, 99213       4
99384              3
99394              3
99395, 99214       2
99203, 99385       2
99443              2
99406, 99203       1
99406              1
99255              1
99396, 99212       1
99242              1
99406, 99205       1
99245              1
99387              1
Name: clean_final, dtype: int64

In [12]:
df_copy.clean_suggested.value_counts()

99213           2148
                1674
99214           1518
99203            922
99212            868
99204            508
99202            194
99396            119
99395            111
99215            102
99386             56
99385             48
99397             35
99205             21
99442             19
99211             15
99406, 99214      14
99387             12
99417, 99215       9
99406, 99204       8
99441              8
99384              3
99406, 99215       1
99406, 99203       1
99203, 99213       1
99417              1
99406, 99213       1
99386, 99396       1
99394              1
Name: clean_suggested, dtype: int64

In [13]:
def get_counts(x):
    
    suggested = str(x['New_4'])
    final = str(x['clean_final'])
    
    if suggested != '':
        suggested_set = set(suggested.split(', '))
        final_set = set(final.split(', '))
        if '9' not in final:
            final_set.clear()    
        if '9' not in suggested:
            suggested_set.clear()
            
        accepted = len(suggested_set.intersection(final_set))
        rejected = len(suggested_set - final_set)
        added = len(final_set - suggested_set)
        
        if (rejected >= 1 or added >= 1):
            print('rejected',rejected)
            print('added',added)
            print('suggested_set',suggested_set,'\t final_set',final_set)
        return pd.Series({'accepted':accepted, 'rejected':rejected, 'added':added})

In [14]:
count_df =  df_copy.apply(get_counts, axis=1)

rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99213'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99204'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99213'} 	 final_set set()
rejected 1
added 0
sugge

In [15]:
df2 = pd.concat([df_copy, count_df], axis = 1)

In [16]:
df2

,encounter_id,service_date,enc_visit_type,enc_place_of_service,facility_code,account_number,cpt_final,cpt_suggested,eandm_final,eandm_suggested,...,eandm_added_count,local_suggested_14_july,new,17_july_head,New_4,clean_suggested,clean_final,accepted,rejected,added
0,8235901,2023-07-06 11:00:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095114,85610,"{85610=[QW], 1036F=[]}",NaN,{},...,0,99212,99212,99212,99212,,,0,1,0
1,8256124,2023-07-06 13:30:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095124,85610,"{1170F=[], 1160F=[], 85610=[QW], 1159F=[], 103...",NaN,{},...,0,99213,99213,99213,99213,,,0,1,0
2,8256117,2023-07-06 13:15:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8803379,85610,"{85610=[QW], 1035F=[], 1034F=[]}",NaN,{},...,0,99212,99212,99212,99212,,,0,1,0
3,8229493,2023-07-05 14:00:00,*Hosp Follow-up 30,Office or Other Outpatient Visit,3038,8057412,"3008F, 93040, 3077F, 1036F, 3078F","{3008F=[], 93040=[], 3077F=[], 1036F=[], 3078F...",99214,{99214=[25]},...,0,99214,99214,99214,99214,99214,99214,1,0,0
4,8234612,2023-07-06 09:30:00,*Follow-up 15,Office or Other Outpatient Visit,3038,8057430,"1160F, 93040, 1159F, 1126F, 1036F","{1160F=[], 93040=[], 1159F=[], 1126F=[], 1036F...",99214,{99214=[25]},...,0,99214,99214,99214,99214,99214,99214,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8414,8228747,2023-07-05 14:00:00,*Follow-up 15,OV,1523,4060723,NaN,{81001=[]},99213,{99213=[]},...,0,99213,99213,99213,99213,99213,99213,1,0,0
8415,8082306,2023-06-20 13:00:00,uroflow/PVR,OV,1523,4329261,"1160F, 1159F, 1036F","{1160F=[], 1159F=[], 1036F=[]}",99214,{99214=[25]},...,0,99214,99214,99214,99214,99214,99214,1,0,0
8416,8213382,2023-07-05 13:30:00,ER FOLLOW-UP,OV,1523,4351334,"1160F, 1159F, 1036F","{3008F=[], 1160F=[], 1159F=[], 87210=[QW], 103...",99213,{99213=[]},...,0,99213,99213,99213,99213,99213,99213,1,0,0
8417,8113030,2023-06-30 11:30:00,ESTABLISHED PATIENT,OV,1523,4349846,81025,{81025=[]},99213,{99213=[]},...,0,99213,99213,99213,99213,99213,99213,1,0,0


In [17]:
df2.eandm_rejected_count.sum()

2272

In [18]:
df2.rejected.sum()

3854

In [19]:
from collections import OrderedDict
def get_accuracy_new(data_df):
    
    # display(data_df)
    # print(data_df.columns)
    
    accept = data_df['accepted'].sum()
    reject = data_df['rejected'].sum()
    add = data_df['added'].sum()
    
    d = OrderedDict()
    d['encounters'] = data_df.shape[0]
    
    d['New'+'_accepted'] = accept
    d['New'+'_rejected'] = reject
    d['New'+'_added'] = add
    
    d['New'+'_billed'] = accept+add
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [20]:
df2.groupby(['facility_code']).apply(get_accuracy_new).sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/test_local.xlsx')

In [ ]:
df2.groupby(['facility_code']).apply(get_accuracy, codetype='eandm').sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/test_with_rule.xlsx')

In [ ]:
new_df = df2.copy()

In [ ]:

new_df.info()


In [ ]:
for i, r in new_df.iterrows():
    if '99024' in str(r['cpt_suggested']) or '0502F' in str(r['cpt_suggested']):
        new_df.at[i, 'accepted'] = 1

In [ ]:
new_df

In [ ]:
new_df.groupby(['facility_code']).apply(get_accuracy_new).sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/after_applying_0502F_and_99024_rule.xlsx')

In [ ]:
new_df.to_excel('')